In [1]:
import time
import torch
import sbibm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import io
import scipy

from pydmd import DMDc
np.set_printoptions(suppress=True)

In [2]:
def create_system_without_B():
    n = 5  # dimension snapshots
    m = 15  # number snapshots
    A = scipy.linalg.helmert(n, True)
    B = np.random.rand(n, n) - .5
    x0 = np.array([0.25] * n)
    u = np.random.rand(n, m - 1) - .5
    snapshots = [x0]
    for i in range(m - 1):
        snapshots.append(A.dot(snapshots[i]) + B.dot(u[:, i]))
    snapshots = np.array(snapshots).T
    return A, B, {'snapshots': snapshots, 'u': u, 'B': B, 'A': A}

In [3]:
def test_reconstruct_b_unknown():
    A, B, system = create_system_without_B()
    dmdc = DMDc(svd_rank=-1, opt=True)
    dmdc.fit(system['snapshots'], system['u'])
    A_hat, B_hat, _ = dmdc.reconstructed_data()
    return (A, B), (A_hat, B_hat)

In [4]:
(A, B), (A_hat, B_hat) = test_reconstruct_b_unknown()

In [5]:
print(f"A = {A}")
print(f"A_hat = {np.real(A_hat)}")

A = [[ 0.4472136   0.4472136   0.4472136   0.4472136   0.4472136 ]
 [ 0.70710678 -0.70710678  0.          0.          0.        ]
 [ 0.40824829  0.40824829 -0.81649658  0.          0.        ]
 [ 0.28867513  0.28867513  0.28867513 -0.8660254   0.        ]
 [ 0.2236068   0.2236068   0.2236068   0.2236068  -0.89442719]]
A_hat = [[ 0.4472136   0.4472136   0.4472136   0.4472136   0.4472136 ]
 [ 0.70710678 -0.70710678  0.          0.          0.        ]
 [ 0.40824829  0.40824829 -0.81649658 -0.         -0.        ]
 [ 0.28867513  0.28867513  0.28867513 -0.8660254   0.        ]
 [ 0.2236068   0.2236068   0.2236068   0.2236068  -0.89442719]]


In [6]:
print(f"B = {B}")
print(f"B_hat = {B_hat}")

B = [[-0.3441854  -0.3894406   0.23998832  0.25990752  0.25210034]
 [-0.08082887 -0.37651072  0.32664513 -0.09309935  0.14101884]
 [ 0.04957299  0.3798759   0.32746753 -0.40025426  0.00704828]
 [-0.38318625  0.26721857  0.48625101  0.49928778  0.38172367]
 [ 0.33538022  0.20711934 -0.26449538  0.39883571 -0.45051171]]
B_hat = [[-0.3441854  -0.3894406   0.23998832  0.25990752  0.25210034]
 [-0.08082887 -0.37651072  0.32664513 -0.09309935  0.14101884]
 [ 0.04957299  0.3798759   0.32746753 -0.40025426  0.00704828]
 [-0.38318625  0.26721857  0.48625101  0.49928778  0.38172367]
 [ 0.33538022  0.20711934 -0.26449538  0.39883571 -0.45051171]]
